In [1]:
# import utility functions
import utils
from utils.flow_models import *
from utils.target import *
from utils.training import *

# import pytorch
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.distributions.transforms as transform
# set number of threads
torch.set_num_threads(8)

# numerical libs
import scipy
import scipy.io

# import plotting
import matplotlib.pyplot as plt
%matplotlib inline

# import I/O
import os
import sys

# set seed
np.random.seed(8)
torch.manual_seed(0)

def loss_func_init(x, prior_logpdf, targ_logpdf):
        """ evaluate initial KL divergence between posterior distribution (NF + prior) 
        and target. x is samples without flow. This is a Monte-Carlo estimation of the 
        log partition function. """
        return (prior_logpdf - targ_logpdf(x)).mean()

## Rank 30 + ResNet

In [ ]:
# load tensorizing flow dataset
double_rosen_tf_dataset = utils.datasets.TensorizingFlowDataset("double_rosen3d_truncated.mat", gaussian_data=False)
# initialize NF model
rosen_flow = NormalizingFlow(dim=double_rosen_tf_dataset.dim, blocks=RESNET_BLOCKS2_DROSEN3, \
                            flow_length=4)

# begin training
rosen_report = train(double_rosen_tf_dataset, rosen_flow, double_rosen_logpdf3d, 
          num_epochs=100,
          batch_size=2**8,
          verbose=True,
          lr=3e-4, 
          use_scheduler=True,
          grad_clip=1e+4)

## Normalizing Flow

In [ ]:
# load tensorizing flow dataset
double_rosen_nf_dataset = utils.datasets.TensorizingFlowDataset("double_rosen3d_truncated.mat", gaussian_data=True)
# initialize NF model
rosen_flow = NormalizingFlow(dim=double_rosen_nf_dataset.dim, blocks=RESNET_BLOCKS2_DROSEN3, \
                            flow_length=4)

# begin training
rosen_nf_report = train(double_rosen_nf_dataset, rosen_flow, double_rosen_logpdf3d, 
          num_epochs=100,
          batch_size=2**8,
          verbose=True,
          lr=3e-4, 
          use_scheduler=True,
          grad_clip=1e+4)

## Compare Training Results

## Plot Training Curve

In [ ]:
plt.figure(1);
plt.plot(rosen_report['test_loss'], label='tensorizing')
plt.plot(rosen_nf_report['test_loss'], label='normalizing')
plt.ylim([14, 20])
plt.legend()

## Compare Samples

In [ ]:
plt.figure(2, figsize=(6, 5));
plt.scatter(rosen_report['post_training_samples'][:, -2], rosen_report['post_training_samples'][:, -1], \
            label='tensor', s=1.2)
plt.scatter(rosen_nf_report['post_training_samples'][:, -2], rosen_nf_report['post_training_samples'][:, -1], \
            label='normal', s=1.2)
plt.legend()

## Full Rank Statistics

In [ ]:
full_rank_data = scipy.io.loadmat('./data/double_rosen3d_full_rank.mat')['X']
full_rank_loglikes = np.log(scipy.io.loadmat('./data/double_rosen3d_full_rank.mat')['likes'])

mean = full_rank_data.mean(1) 
std = full_rank_data.std(1)
# compute KL divergence before NF training
opt_kl = loss_func_init(torch.Tensor(full_rank_data.T),\
                        torch.Tensor(full_rank_loglikes), double_rosen_logpdf3d)

opt_kl = opt_kl.item()

In [ ]:
# save exact stats
double_rosen3 = {'mean': mean, 'std': std, 'loss': opt_kl}
scipy.io.savemat('./utils/full_rank_stats5.mat', {'double_rosen3': double_rosen3})

In [2]:
# delete
full_rank_data = scipy.io.loadmat('./data/gl1d_spin_glass_new_rank1.mat')['X']
full_rank_loglikes = np.log(scipy.io.loadmat('./data/gl1d_spin_glass_new_rank1.mat')['likes'])

mean = full_rank_data.mean(1) 
std = full_rank_data.std(1)
# compute KL divergence before NF training
opt_kl = loss_func_init(torch.Tensor(full_rank_data.T),\
                        torch.Tensor(full_rank_loglikes), ginzburg_landau1d_spin_logpdf)

opt_kl = opt_kl.item()

In [3]:
opt_kl

-9.296208484449766

In [7]:
opt_kl

-56.26873108842776